# Database Reset and Reload Script

This notebook resets and reloads the project database from the Excel file.

## Multi-Phase Support

The script now supports all three project phases:
- **Fase I - Prefactibilidad**: Uses `item_fase_i` table
- **Fase II - Factibilidad**: Uses `item_fase_ii` table (excludes subcomponents of GEOLOGÍA and TALUDES)
- **Fase III - Diseños a detalle**: Uses `item_fase_iii` table

Each phase has different budget item structures, and the data will be automatically loaded into the correct table based on the project's FASE field.


In [ ]:
import sqlite3
import os
import sys

# Add project root to path (for Jupyter notebooks)
# Get the current directory and navigate to project root
project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
sys.path.insert(0, project_root)

from app.config import Config
from app.services import EDA

%load_ext autoreload
%autoreload 2
%reload_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
def drop_all_tables() -> None:
    """Drop all existing tables in the database."""
    db = sqlite3.connect(Config.OLD_DATABASE)
    cursor = db.cursor()
    # Drop children first due to foreign keys
    cursor.execute('DROP TABLE IF EXISTS item')  # Old table (if exists)
    cursor.execute('DROP TABLE IF EXISTS item_fase_i')
    cursor.execute('DROP TABLE IF EXISTS item_fase_ii')
    cursor.execute('DROP TABLE IF EXISTS item_fase_iii')
    cursor.execute('DROP TABLE IF EXISTS unidad_funcional')
    cursor.execute('DROP TABLE IF EXISTS proyectos')
    db.commit()
    db.close()

In [ ]:
def create_database() -> None:
    db = sqlite3.connect(Config.OLD_DATABASE)
    db.execute('''
        CREATE TABLE IF NOT EXISTS proyectos (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            nombre TEXT NOT NULL,
            codigo TEXT UNIQUE NOT NULL,
            num_ufs INTEGER,
            longitud REAL,
            anio_inicio INTEGER,
            duracion INTEGER,
            fase TEXT,
            ubicacion TEXT,
            costo REAL,
            lat_inicio REAL,
            lng_inicio REAL,
            lat_fin REAL,
            lng_fin REAL,
            created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
        )
    ''')
    
    db.execute('''
        CREATE TABLE IF NOT EXISTS unidad_funcional (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            codigo TEXT NOT NULL,
            unidad_funcional INTEGER NOT NULL,
            longitud_km REAL,
            puentes_vehiculares_und INTEGER,
            puentes_vehiculares_mt2 INTEGER,
            puentes_peatonales_und INTEGER,
            puentes_peatonales_mt2 INTEGER,
            tuneles_und INTEGER,
            tuneles_km REAL,
            alcance TEXT,
            zona TEXT,
            tipo_terreno TEXT,
            FOREIGN KEY (codigo) REFERENCES proyectos(codigo) ON DELETE CASCADE
        )
    ''')
    
    # Fase I - Prefactibilidad
    db.execute('''
        CREATE TABLE IF NOT EXISTS item_fase_i (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            codigo TEXT NOT NULL UNIQUE,
            transporte REAL DEFAULT 0,
            diseno_geometrico REAL DEFAULT 0,
            prefactibilidad_tuneles REAL DEFAULT 0,
            geologia REAL DEFAULT 0,
            geotecnia REAL DEFAULT 0,
            hidrologia_hidraulica REAL DEFAULT 0,
            ambiental_social REAL DEFAULT 0,
            predial REAL DEFAULT 0,
            riesgos_sostenibilidad REAL DEFAULT 0,
            evaluacion_economica REAL DEFAULT 0,
            gestion_predial REAL DEFAULT 0,
            socioeconomica_financiera REAL DEFAULT 0,
            estructuras REAL DEFAULT 0,
            direccion_coordinacion REAL DEFAULT 0,
            FOREIGN KEY (codigo) REFERENCES proyectos(codigo) ON DELETE CASCADE
        )
    ''')
    
    # Fase II - Factibilidad
    db.execute('''
        CREATE TABLE IF NOT EXISTS item_fase_ii (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            codigo TEXT NOT NULL UNIQUE,
            transporte REAL DEFAULT 0,
            topografia REAL DEFAULT 0,
            geologia REAL DEFAULT 0,
            taludes REAL DEFAULT 0,
            hidrologia_hidraulica REAL DEFAULT 0,
            estructuras REAL DEFAULT 0,
            tuneles REAL DEFAULT 0,
            pavimento REAL DEFAULT 0,
            predial REAL DEFAULT 0,
            ambiental_social REAL DEFAULT 0,
            costos_presupuestos REAL DEFAULT 0,
            socioeconomica REAL DEFAULT 0,
            direccion_coordinacion REAL DEFAULT 0,
            FOREIGN KEY (codigo) REFERENCES proyectos(codigo) ON DELETE CASCADE
        )
    ''')
    
    # Fase III - Detalle (original item table)
    db.execute('''
        CREATE TABLE IF NOT EXISTS item_fase_iii (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            codigo TEXT NOT NULL UNIQUE,
            transporte REAL DEFAULT 0,
            informacion_geografica REAL DEFAULT 0,
            trazado_diseno_geometrico REAL DEFAULT 0,
            seguridad_vial REAL DEFAULT 0,
            sistemas_inteligentes REAL DEFAULT 0,
            geologia REAL DEFAULT 0,
            hidrogeologia REAL DEFAULT 0,
            suelos REAL DEFAULT 0,
            taludes REAL DEFAULT 0,
            pavimento REAL DEFAULT 0,
            socavacion REAL DEFAULT 0,
            estructuras REAL DEFAULT 0,
            tuneles REAL DEFAULT 0,
            urbanismo_paisajismo REAL DEFAULT 0,
            predial REAL DEFAULT 0,
            impacto_ambiental REAL DEFAULT 0,
            cantidades REAL DEFAULT 0,
            evaluacion_socioeconomica REAL DEFAULT 0,
            otros_manejo_redes REAL DEFAULT 0,
            direccion_coordinacion REAL DEFAULT 0,
            FOREIGN KEY (codigo) REFERENCES proyectos(codigo) ON DELETE CASCADE
        )
    ''')
    
    db.commit()
    
    db.close()

In [10]:
drop_all_tables()
create_database()

In [ ]:
filename = "../data/BASE DE DATOS PRESUPUESTOS.xlsx"
preproccesing = EDA(filename)
df = preproccesing.assemble_projects_from_excel()
df

,NOMBRE DEL PROYECTO,CÓDIGO DEL PROYECTO,AÑO INICIO,FASE,DEPARTAMENTO,LONGITUD KM,PUENTES VEHICULARES UND,PUENTES VEHICULARES M2,PUENTES PEATONALES UND,PUENTES PEATONALES M2,...,7 - SOCAVACIÓN,8 - ESTRUCTURAS,9 - TÚNELES,10 - URBANISMO Y PAISAJISMO,11 - PREDIAL,12 - IMPACTO AMBIENTAL,13 - CANTIDADES,14 - EVALUACIÓN SOCIOECONÓMICA,15 - OTROS - MANEJO DE REDES,16 - DIRECCIÓN Y COORDINACIÓN
0,AUTOPISTAS DE LA MONTAÑA,45000036221,2010,Fase II - Factibilidad,ANTIOQUIA - CALDAS Y RISARALDA,26.200,14.0,4138.56000,0.0,0.000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,AUTOPISTAS DE LA MONTAÑA,45000036221,2010,Fase II - Factibilidad,ANTIOQUIA - CALDAS Y RISARALDA,23.800,19.0,10311.40000,0.0,0.000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,AUTOPISTAS DE LA MONTAÑA,45000036221,2010,Fase II - Factibilidad,ANTIOQUIA - CALDAS Y RISARALDA,24.900,7.0,1897.26000,0.0,0.000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,AUTOPISTAS DE LA MONTAÑA,45000036221,2010,Fase II - Factibilidad,ANTIOQUIA - CALDAS Y RISARALDA,14.600,9.0,8428.80000,0.0,0.000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,AUTOPISTAS DE LA MONTAÑA,45000036221,2010,Fase II - Factibilidad,ANTIOQUIA - CALDAS Y RISARALDA,49.400,33.0,11141.76000,0.0,0.000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,AUTOPISTAS DE LA MONTAÑA,45000036221,2010,Fase II - Factibilidad,ANTIOQUIA - CALDAS Y RISARALDA,40.800,67.0,47596.30000,0.0,0.000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,AUTOPISTAS DE LA MONTAÑA,45000036221,2010,Fase II - Factibilidad,ANTIOQUIA - CALDAS Y RISARALDA,26.400,40.0,39907.20000,0.0,0.000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,CATAMBUCO - PASTO,0602301,2024,Fase I - Prefactibilidad,Nariño,5.200,2.0,1816.80000,7.0,878.990,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,PEDREGAL - PASTO UF4-UF5,6935,2016,Fase III - Diseños a detalle,Nariño,15.760,4.0,6292.00000,0.0,0.000,...,7.078101e+07,2.261120e+08,0.0,1.612921e+07,0.0,0.00,0.000000e+00,0.0,1.171310e+08,1.042543e+08
9,PEDREGAL - PASTO UF4-UF5,6935,2016,Fase III - Diseños a detalle,Nariño,22.200,0.0,0.00000,1.0,77.000,...,7.078101e+07,2.261120e+08,0.0,1.612921e+07,0.0,0.00,0.000000e+00,0.0,1.171310e+08,1.042543e+08


In [ ]:

import pandas as pd

# Connect to database
db = sqlite3.connect(Config.OLD_DATABASE)
cursor = db.cursor()

# Drop and recreate tables
drop_all_tables()
create_database()

# Build projects dictionary
print("\nProcessing projects...")
proyectos_dict = {}

# Excel column name -> Database field name mappings (matching what eda.py reads)
# Fase I - Prefactibilidad (13 items)
item_columns_fase_i = {
    '1 - TRANSPORTE': 'transporte',
    '2 - DISEÑO GEOMÉTRICO': 'diseno_geometrico',
    '3 - PREFACTIBILIDAD TÚNELES': 'prefactibilidad_tuneles',
    '4 - GEOLOGIA': 'geologia',
    '5 - GEOTECNIA': 'geotecnia',
    '6 - HIDROLOGÍA E HIDRÁULICA': 'hidrologia_hidraulica',
    '7 - AMBIENTAL Y SOCIAL': 'ambiental_social',
    '8 - PREDIAL': 'predial',
    '9 - RIESGOS Y SOSTENIBILIDAD': 'riesgos_sostenibilidad',
    '10 - EVALUACIÓN ECONÓMICA': 'evaluacion_economica',
    '11 - SOCIO ECONÓMICA, FINANCIERA': 'socioeconomica_financiera',
    '12 - ESTRUCTURAS': 'estructuras',
    '13 - DIRECCIÓN Y COORDINACIÓN': 'direccion_coordinacion'
}

# Fase II - Factibilidad (17 items in Excel, aggregate subcomponents into 13 DB fields)
item_columns_fase_ii = {
    '1 - TRANSPORTE': 'transporte',
    '2 - TRAZADO Y TOPOGRAFIA': 'topografia',
    '2.1 - INFORMACIÓN GEOGRÁFICA': 'topografia',  # Add to main item
    '2.2 - TRAZADO Y DISEÑO GEOMÉTRICO': 'topografia',  # Add to main item
    '3 - GEOLOGÍA': 'geologia',
    '3.1 - GEOLOGÍA': 'geologia',  # Add to main item
    '3.2 - HIDROGEOLOGÍA': 'geologia',  # Add to main item
    '4 - TALUDES': 'taludes',
    '5 - HIDROLOGÍA E HIDRÁULICA': 'hidrologia_hidraulica',
    '6 - ESTRUCTURAS': 'estructuras',
    '7 - TÚNELES': 'tuneles',
    '8 - PAVIMENTO': 'pavimento',
    '9 - PREDIAL': 'predial',
    '10 - AMBIENTAL Y SOCIAL': 'ambiental_social',
    '11 - COSTOS Y PRESUPUESTOS': 'costos_presupuestos',
    '12 - SOCIOECONÓMICA': 'socioeconomica',
    '13 - DIRECCIÓN Y COORDINACIÓN': 'direccion_coordinacion'
}

# Fase III - Diseños a detalle (22 items in Excel, 20 DB fields - skip parent headers)
item_columns_fase_iii = {
    '1 - TRANSPORTE': 'transporte',
    '2 - TRAZADO Y DISEÑO GEOMÉTRICO': None,  # Parent header - skip
    '2.1 - INFORMACIÓN GEOGRÁFICA': 'informacion_geografica',
    '2.2 TRAZADO Y DISEÑO GEOMÉTRICO': 'trazado_diseno_geometrico',
    '2.3 - SEGURIDAD VIAL': 'seguridad_vial',
    '2.4 - SISTEMAS INTELIGENTES': 'sistemas_inteligentes',
    '3 - GEOLOGÍA': None,  # Parent header - skip
    '3.1 - GEOLOGÍA': 'geologia',
    '3.2 - HIDROGEOLOGÍA': 'hidrogeologia',
    '4 - SUELOS': 'suelos',
    '5 - TALUDES': 'taludes',
    '6 - PAVIMENTO': 'pavimento',
    '7 - SOCAVACIÓN': 'socavacion',
    '8 - ESTRUCTURAS': 'estructuras',
    '9 - TÚNELES': 'tuneles',
    '10 - URBANISMO Y PAISAJISMO': 'urbanismo_paisajismo',
    '11 - PREDIAL': 'predial',
    '12 - IMPACTO AMBIENTAL': 'impacto_ambiental',
    '13 - CANTIDADES': 'cantidades',
    '14 - EVALUACIÓN SOCIOECONÓMICA': 'evaluacion_socioeconomica',
    '15 - OTROS - MANEJO DE REDES': 'otros_manejo_redes',
    '16 - DIRECCIÓN Y COORDINACIÓN': 'direccion_coordinacion'
}

# Function to determine which mapping to use based on FASE
def get_item_mapping(fase):
    if 'Fase I - Prefactibilidad' in fase:
        return item_columns_fase_i, 'item_fase_i'
    elif 'Fase II - Factibilidad' in fase:
        return item_columns_fase_ii, 'item_fase_ii'
    else:  # Fase III or default
        return item_columns_fase_iii, 'item_fase_iii'

for idx, row in df.iterrows():
    codigo = str(row['CÓDIGO DEL PROYECTO']).strip()

    if codigo not in proyectos_dict:
        # First time seeing this project - store project-level data
        fase = row['FASE'] if pd.notna(row['FASE']) else 'Fase III - Diseños a detalle'
        
        # Get the appropriate mapping for this project's fase
        item_columns_mapping, table_name = get_item_mapping(fase)

        # Collect and aggregate item values
        proyecto_items = {}
        for excel_col, db_field in item_columns_mapping.items():
            if db_field is None:  # Skip parent headers in Fase III
                continue
            if excel_col in df.columns:
                valor = float(row[excel_col]) if pd.notna(row[excel_col]) else 0.0
                # For Fase II, aggregate subcomponents into main items
                if db_field in proyecto_items:
                    proyecto_items[db_field] += valor
                else:
                    proyecto_items[db_field] = valor
        
        costo_total = sum(proyecto_items.values())

        proyectos_dict[codigo] = {
            'nombre': row['NOMBRE DEL PROYECTO'],
            'codigo': codigo,
            'anio_inicio': int(row['AÑO INICIO']) if pd.notna(row['AÑO INICIO']) else None,
            'fase': fase,
            'ubicacion': row['DEPARTAMENTO'] if pd.notna(row['DEPARTAMENTO']) else None,
            'num_ufs': 0,
            'longitud': 0.0,
            'costo': round(costo_total, 2),
            'unidades': [],
            'items_totals': proyecto_items,
            'table_name': table_name
        }
        print(f"Project {codigo} ({fase}): ${costo_total:,.2f}")

    # Track functional units and aggregate metrics
    proyecto = proyectos_dict[codigo]
    proyecto['num_ufs'] += 1
    proyecto['longitud'] += float(row['LONGITUD KM']) if pd.notna(row['LONGITUD KM']) else 0.0

    # Store functional unit data
    uf_data = {
        'unidad_funcional': proyecto['num_ufs'],  # Sequential number
        'longitud_km': float(row['LONGITUD KM']) if pd.notna(row['LONGITUD KM']) else 0.0,
        'puentes_vehiculares_und': int(row['PUENTES VEHICULARES UND']) if pd.notna(row['PUENTES VEHICULARES UND']) else 0,
        'puentes_vehiculares_mt2': int(row['PUENTES VEHICULARES M2']) if pd.notna(row['PUENTES VEHICULARES M2']) else 0,
        'puentes_peatonales_und': int(row['PUENTES PEATONALES UND']) if pd.notna(row['PUENTES PEATONALES UND']) else 0,
        'puentes_peatonales_mt2': int(row['PUENTES PEATONALES M2']) if pd.notna(row['PUENTES PEATONALES M2']) else 0,
        'tuneles_und': int(row['TUNELES UND']) if pd.notna(row['TUNELES UND']) else 0,
        'tuneles_km': float(row['TUNELES KM']) if pd.notna(row['TUNELES KM']) else 0.0,
        'alcance': row['ALCANCE'] if pd.notna(row['ALCANCE']) else None,
        'zona': row['ZONA'] if pd.notna(row['ZONA']) else None,
        'tipo_terreno': row['TIPO TERRENO'] if pd.notna(row['TIPO TERRENO']) else None,
    }
    proyectos_dict[codigo]['unidades'].append(uf_data)

print(f"✓ Found {len(proyectos_dict)} unique projects")

# 3. Insert projects
print("\nInserting projects into database...")
for codigo, proyecto in proyectos_dict.items():
    cursor.execute('''
        INSERT INTO proyectos (nombre, codigo, num_ufs, longitud, anio_inicio, duracion, fase, ubicacion, costo, lat_inicio, lng_inicio, lat_fin, lng_fin)
        VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
    ''', (
        proyecto['nombre'],
        proyecto['codigo'],
        proyecto['num_ufs'],
        round(proyecto['longitud'], 2),
        proyecto['anio_inicio'],
        None,  # duracion - not in dataframe
        proyecto['fase'],
        proyecto['ubicacion'],
        proyecto['costo'],
        None, None, None, None  # lat/lng - not in dataframe yet
    ))

db.commit()
print(f"✓ Inserted {len(proyectos_dict)} projects")

# 4. Insert functional units
print("\nInserting functional units...")
uf_count = 0
for codigo, proyecto in proyectos_dict.items():
    for uf in proyecto['unidades']:
        cursor.execute('''
            INSERT INTO unidad_funcional (codigo, unidad_funcional, longitud_km, puentes_vehiculares_und, puentes_vehiculares_mt2,
                                         puentes_peatonales_und, puentes_peatonales_mt2, tuneles_und, tuneles_km, alcance, zona, tipo_terreno)
            VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
        ''', (
            codigo,
            uf['unidad_funcional'],
            uf['longitud_km'],
            uf['puentes_vehiculares_und'],
            uf['puentes_vehiculares_mt2'],
            uf['puentes_peatonales_und'],
            uf['puentes_peatonales_mt2'],
            uf['tuneles_und'],
            uf['tuneles_km'],
            uf['alcance'],
            uf['zona'],
            uf['tipo_terreno']
        ))
        uf_count += 1

db.commit()
print(f"✓ Inserted {uf_count} functional units")

# 5. Insert items (one row per project with all item columns)
print("\nInserting items...")
item_count = 0
fase_counts = {'item_fase_i': 0, 'item_fase_ii': 0, 'item_fase_iii': 0}

for codigo, proyecto in proyectos_dict.items():
    # Get all item values, defaulting to 0 if not present
    items_data = proyecto['items_totals']
    table_name = proyecto['table_name']
    fase_counts[table_name] += 1
    
    if table_name == 'item_fase_i':
        # Insert into Fase I table (13 fields)
        cursor.execute(f'''
            INSERT INTO {table_name} (
                codigo, transporte, diseno_geometrico, prefactibilidad_tuneles, 
                geologia, geotecnia, hidrologia_hidraulica, ambiental_social, 
                predial, riesgos_sostenibilidad, evaluacion_economica,
                socioeconomica_financiera, estructuras, direccion_coordinacion
            )
            VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
        ''', (
            codigo,
            round(items_data.get('transporte', 0), 2),
            round(items_data.get('diseno_geometrico', 0), 2),
            round(items_data.get('prefactibilidad_tuneles', 0), 2),
            round(items_data.get('geologia', 0), 2),
            round(items_data.get('geotecnia', 0), 2),
            round(items_data.get('hidrologia_hidraulica', 0), 2),
            round(items_data.get('ambiental_social', 0), 2),
            round(items_data.get('predial', 0), 2),
            round(items_data.get('riesgos_sostenibilidad', 0), 2),
            round(items_data.get('evaluacion_economica', 0), 2),
            round(items_data.get('socioeconomica_financiera', 0), 2),
            round(items_data.get('estructuras', 0), 2),
            round(items_data.get('direccion_coordinacion', 0), 2)
        ))
    elif table_name == 'item_fase_ii':
        # Insert into Fase II table
        cursor.execute(f'''
            INSERT INTO {table_name} (
                codigo, transporte, topografia, geologia, taludes, 
                hidrologia_hidraulica, estructuras, tuneles, pavimento, 
                predial, ambiental_social, costos_presupuestos, socioeconomica, 
                direccion_coordinacion
            )
            VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
        ''', (
            codigo,
            round(items_data.get('transporte', 0), 2),
            round(items_data.get('topografia', 0), 2),
            round(items_data.get('geologia', 0), 2),
            round(items_data.get('taludes', 0), 2),
            round(items_data.get('hidrologia_hidraulica', 0), 2),
            round(items_data.get('estructuras', 0), 2),
            round(items_data.get('tuneles', 0), 2),
            round(items_data.get('pavimento', 0), 2),
            round(items_data.get('predial', 0), 2),
            round(items_data.get('ambiental_social', 0), 2),
            round(items_data.get('costos_presupuestos', 0), 2),
            round(items_data.get('socioeconomica', 0), 2),
            round(items_data.get('direccion_coordinacion', 0), 2)
        ))
    else:  # item_fase_iii
        # Insert into Fase III table
        cursor.execute(f'''
            INSERT INTO {table_name} (
                codigo, transporte, informacion_geografica, trazado_diseno_geometrico, 
                seguridad_vial, sistemas_inteligentes, geologia, hidrogeologia, suelos, 
                taludes, pavimento, socavacion, estructuras, tuneles, urbanismo_paisajismo, 
                predial, impacto_ambiental, cantidades, evaluacion_socioeconomica, 
                otros_manejo_redes, direccion_coordinacion
            )
            VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
        ''', (
            codigo,
            round(items_data.get('transporte', 0), 2),
            round(items_data.get('informacion_geografica', 0), 2),
            round(items_data.get('trazado_diseno_geometrico', 0), 2),
            round(items_data.get('seguridad_vial', 0), 2),
            round(items_data.get('sistemas_inteligentes', 0), 2),
            round(items_data.get('geologia', 0), 2),
            round(items_data.get('hidrogeologia', 0), 2),
            round(items_data.get('suelos', 0), 2),
            round(items_data.get('taludes', 0), 2),
            round(items_data.get('pavimento', 0), 2),
            round(items_data.get('socavacion', 0), 2),
            round(items_data.get('estructuras', 0), 2),
            round(items_data.get('tuneles', 0), 2),
            round(items_data.get('urbanismo_paisajismo', 0), 2),
            round(items_data.get('predial', 0), 2),
            round(items_data.get('impacto_ambiental', 0), 2),
            round(items_data.get('cantidades', 0), 2),
            round(items_data.get('evaluacion_socioeconomica', 0), 2),
            round(items_data.get('otros_manejo_redes', 0), 2),
            round(items_data.get('direccion_coordinacion', 0), 2)
        ))
    item_count += 1

db.commit()
print(f"✓ Inserted {item_count} item records (1 per project)")
print(f"  - Fase I (Prefactibilidad): {fase_counts['item_fase_i']} projects")
print(f"  - Fase II (Factibilidad): {fase_counts['item_fase_ii']} projects")
print(f"  - Fase III (Diseños a detalle): {fase_counts['item_fase_iii']} projects")

# Close connection
db.close()

print("\n" + "="*60)
print("DATABASE POPULATION COMPLETED SUCCESSFULLY!")
print("="*60)
print(f"Total Projects: {len(proyectos_dict)}")
print(f"Total Functional Units: {uf_count}")
print(f"Total Cost: ${sum(p['costo'] for p in proyectos_dict.values()):,.2f}")
print("="*60)



Processing projects...
Project 45000036221 (Fase II - Factibilidad): $1,238,647,591.00
Project 0602301 (Fase I - Prefactibilidad): $99,598,985.38
Project 6935 (Fase III - Diseños a detalle): $862,658,079.71
Project 0654801 (Fase III - Diseños a detalle): $3,729,730,912.08
Project 0581301 (Fase III - Diseños a detalle): $610,090,248.00
Project 0001 (Fase III - Diseños a detalle): $893,560,504.30
Project 0552903 (Fase III - Diseños a detalle): $371,076,561.00
Project 0266702 (Fase III - Diseños a detalle): $68,672,568.26
Project 0004 (Fase III - Diseños a detalle): $432,039,843.00
Project 0427901 (Fase III - Diseños a detalle): $54,125,203.96
Project 0321501 (Fase III - Diseños a detalle): $824,025,521.66
Project 0347801 (Fase III - Diseños a detalle): $2,807,099,464.10
Project 0300605 (Fase III - Diseños a detalle): $295,481,099.12
Project 0300604 (Fase III - Diseños a detalle): $43,016,606.22
Project 0005 (Fase III - Diseños a detalle): $268,109,504.19
Project 445501 (Fase II - Factib